In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import gc
import sys

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a=1

In [ ]:
# folder_path = r"H:\phd stuff\tidy3d\data\08_23_2024 chi 0.3 N1000 Sample Beam Spreading\z_incidence"
folder_path = r"H:\phd stuff\tidy3d\data\08_14_2024 chi 0.3 N1000 Sample Beam Spreading\z_incidence"
# folder_path = r"H:\phd stuff\tidy3d\data\08_21_2024 chi 0.3 N1000 Sample Beam Spreading\z_incidence"
# file = "Beam Spreading 0.12 - 0.15 - Sample_2 20ps.txt"
file = "Beam Spreading 0.28 - 0.31 - Sample_2 50ps.txt"
data = []
for i,filename in enumerate(natsorted(os.listdir(folder_path))):
    # if filename == file:
        file=os.path.join(folder_path, filename)
        sim_result = (AM.loadFromFile(key = tidy3dAPI, file_path=file))
        lambdas = 1/sim_result.monitor_lambdas
        data += [{"data":sim_result,"lambdas":lambdas}]

In [ ]:
lambdas

In [ ]:
def create_movie(field_time_out, monitor_lambdas,name='',type='t',log=False,path="",frames_per_second=1,rem_frames=False, normalize=True, a=5/3):
    frames = []

    max_values = field_time_out.max(axis=(0,1,2)) if normalize else 1

    field_time_out = field_time_out/max_values
    field_time_out['x'] = field_time_out['x']/a
    field_time_out['y'] = field_time_out['y']/a
    field_time_out['z'] = field_time_out['z']/a


    if log:
        field_log = np.log10((field_time_out))
        folder_pics = "logPics"
        
    else:
        field_log = ((field_time_out))
        folder_pics = "linPics"

    if not os.path.exists(f'{path}/{folder_pics}'):
            os.makedirs(f'{path}/{folder_pics}')
            print(f"Folder {path}/{folder_pics} created successfully.")

    for i, time in enumerate(field_time_out.t):
        if os.path.isfile(f'{path}/{folder_pics}/frame_{i}.png'):
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
            continue
        try:
            fig, ax = plt.subplots(figsize=(14/2, 18/2))
            #fig, ax = plt.subplots(figsize=(12, 6))
            if type=="t":
                pcolormesh = (field_log).isel(t=i).squeeze().plot.pcolormesh(ax=ax,cmap="jet")
            else:
                pcolormesh = (field_log).isel(f=i).squeeze().plot.pcolormesh(ax=ax,cmap="jet")

            ax.set_aspect('auto', adjustable='box')
            #plt.ylim(-50,50)
            try:
                plt.title(f'Time: {str(np.array(field_time_out['t'][()][i])*1e12)} ps')
            except:
                plt.title(f'$\\nu$: {(1/np.array(td.C_0/field_time_out['f'][()][i])):.4g}')


            # Save the frame
            plt.savefig(f'{path}/{folder_pics}/frame_{i}.png')
            plt.close(fig)
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
        except:
            break
        

    name_movie = f'{path}/{name}.mp4' if name else f'output/anderson/d(t) analysis/Diameter d(t) at output of the structure Range - {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}.mp4'
    with imageio.get_writer(name_movie, fps=frames_per_second) as writer:
        for frame in frames:
            image = imageio.imread(frame)
            writer.append_data(image)

    # Optionally, remove the individual frames if no longer needed
    if rem_frames:
        for frame in frames:
            os.remove(frame)
    
    return False

In [ ]:
for item in data:
    item["field_time_out"]=item["data"].sim_data.get_intensity("time_monitorFieldOut")
    item["field"] = item["field_time_out"][:,np.where(np.bool_(item["field_time_out"]["y"]>=-5) & np.bool_(item["field_time_out"]["y"]<=5))[0],:,:]

In [ ]:
# font = {
#         'weight' : 'bold',
#         'size'   : 6}

# matplotlib.rc('font', **font)
# create_movie(field_time_out,a/lambdas,name=f"timemonitor {lambdas[0]:.3g} - {lambdas[-1]:.3g}",type="t",log=False,path=r"H:\phd stuff\tidy3d\output/2D SHU/Beam Spreading Chi 0.3 N10000",frames_per_second=10,normalize=False,a=a)

In [ ]:
# create_movie(field_time_out,a/lambdas,name=f"timemonitor {lambdas[0]:.3g} - {lambdas[-1]:.3g}",type="t",log=True,path=r"H:\phd stuff\tidy3d\output/2D SHU/Beam Spreading Chi 0.3 N10000",frames_per_second=10,normalize=False,a=a)


In [ ]:
# fig, ax = plt.subplots(figsize=(5, 10))
# pcolormesh = (field).isel(t=33).squeeze().plot.pcolormesh(ax=ax,cmap="jet")
# plt.title(f'Time: {str(np.array(field['t'][()][33])*1e12)} ps')
# plt.show()

In [ ]:
import scipy.integrate


def PR(intensity_array):
    x,y = intensity_array['x'],intensity_array['y']
    intensity = intensity_array.values
    integral_1 = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity, x=y, axis=1), x=x, axis=0)**2
    integral_2 = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity**2, x=y, axis=1), x=x, axis=0)

    return ((integral_1/integral_2))

In [ ]:
def calculate_sigma(intensity_array):
    x1,y1 = intensity_array['x'],intensity_array['y']
    x,y = np.meshgrid(x1,y1)
    rho_squared = x**2 + y**2
    sigma_squared=[]
    for t in range(np.shape(intensity_array)[2]):
        denominator = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity_array[:,:,t], x=y1, axis=1), x=x1, axis=0)
        numerator = scipy.integrate.trapezoid(scipy.integrate.trapezoid(rho_squared.T*intensity_array[:,:,t], x=y1, axis=1), x=x1, axis=0)
        sigma_squared += [numerator / denominator]
    
    return sigma_squared

In [ ]:
for item in data:

    item["PR_t"]= (PR(item["field"]).squeeze())

In [ ]:
for item in data:
    item["sigma"] = (calculate_sigma(item["field"][:,:,0,:]))

In [ ]:
def moving_average(x, w=3):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
plt.figure(figsize=(15, 10))

for item in data:
    # plt.plot((item["field_time_out"]["t"]*1e12)[2:],moving_average(item["PR_t"],3), label=rf"$\nu$ = {item["lambdas"][0]:.2g} - {item["lambdas"][1]:.2g}")
    plt.plot((item["field_time_out"]["t"]*1e12),(item["PR_t"]), label=rf"$\nu$ = {item["lambdas"][0]:.2g} - {item["lambdas"][1]:.2g}")


plt.ylabel("PR(t)")
plt.xlabel("Time[ps]")
plt.title("PR(t) at Diffusive and Localized Regimes")
# plt.ylim(0,150)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

for item in data:
    plt.plot(item["field_time_out"]["t"]*1e12,item["sigma"])
plt.show()